In [ ]:
# Import necessary libraries
import dotenv
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download the necessary NLTK data
nltk.download("punkt")
nltk.download("stopwords")

dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_PLACES_API_KEY")
OPEN_AI_API_KEY = os.getenv("OPEN_AI_API_KEY")


# Function to remove stop words, punctuation, and double quotes
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove double quotes
    text = text.replace('"', "")
    # Tokenize the text into words
    words = word_tokenize(text)
    # Remove punctuation
    words = [word for word in words if word.isalnum()]
    # Remove stop words
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]
    # Rejoin words into a single string
    cleaned_text = " ".join(filtered_words)
    return cleaned_text


# Prompt the user for a string
user_input = input("Enter a string: ")
# Clean the input string
cleaned_text = clean_text(user_input)

# Print the cleaned string
print("Cleaned string:", cleaned_text)

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=OPEN_AI_API_KEY)


response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "user",
            "content": f"Create an itinerary and provide only the names for places listed in google places for breakfast, morning activity, lunch, afternoon activity, dinner and evening activity (return only the day and the name): {cleaned_text}",
        }
    ],
    temperature=1,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)
print(response)

# Extract token usage from the response
completion_tokens = response.usage.completion_tokens
prompt_tokens = response.usage.prompt_tokens
total_tokens = completion_tokens + prompt_tokens

# Cost estimation for 1,000,000 requests
total_requests = 1_000_000
total_tokens_for_all_requests = total_tokens * total_requests

training_cost_per_million_tokens = 8.00
input_usage_cost_per_million_tokens = 3.00
output_usage_cost_per_million_tokens = 6.00

total_cost = (total_tokens_for_all_requests / 1_000_000) * (
    training_cost_per_million_tokens
    + input_usage_cost_per_million_tokens
    + output_usage_cost_per_million_tokens
)

print(f"Estimated total cost for {total_requests} requests: ${total_cost:.2f}")

In [ ]:
import pandas as pd

# Extract the itinerary content from the response
itinerary_content = response.choices[0].message.content

# Split the content into days and activities
days = itinerary_content.split("\n\n")
data = []
for day in days:
    lines = day.split("\n")
    day_number = lines[0].split(" ")[1]
    for line in lines[1:]:
        activity_type, place = line.split(": ")
        data.append({"Day": day_number, "Activity Type": activity_type, "Place": place})

# Create a DataFrame from the data
df = pd.DataFrame(data)

# Print the DataFrame
print(df)

In [ ]:
import pandas as pd
import requests


def search_google_places(place, location, api_key):
    # Construct the URL for the Google Places API
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": place,
        "inputtype": "textquery",
        "fields": "name,place_id",
        "locationbias": f"point:{location}",
        "key": api_key,
    }
    # Make the API request
    response = requests.get(base_url, params=params)
    # Parse the response
    if response.status_code == 200:
        result = response.json()
        if result.get("candidates"):
            return result["candidates"][0]["name"], result["candidates"][0]["place_id"]
    return None, None


# API key for Google Places API
api_key = GOOGLE_API_KEY


# Location for the search
location = "25.790654,-80.1300455"  # Latitude and longitude for South Beach Miami, FL

# Create new columns for place name and place ID
df["Place Name"] = ""
df["Place ID"] = ""

# Iterate over the DataFrame and search for each place
for index, row in df.iterrows():
    place_name, place_id = search_google_places(row["Place"], location, api_key)
    df.at[index, "Place Name"] = place_name
    df.at[index, "Place ID"] = place_id

# Print the updated DataFrame
print(df)

In [ ]:
import requests
import pandas as pd

# Assuming `df` is your DataFrame with 'Place Name' and 'Place ID' columns
url_template = "https://places.googleapis.com/v1/places/{}/?fields=id,displayName,location,primaryType,editorialSummary,websiteUri,googleMapsUri,formattedAddress,businessStatus,userRatingCount,priceLevel,rating,goodForChildren&key={api_key}"


def fetch_place_details(place_id):
    url = url_template.format(place_id)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None


# Iterate over each row in the DataFrame and fetch place details
for index, row in df.iterrows():
    place_id = row["Place ID"]  # Corrected column name
    place_details = fetch_place_details(place_id)
    if place_details:
        # Process the JSON data as needed, for example, you can print it
        print(place_details)
    else:
        print(f"Failed to fetch details for place with ID: {place_id}")

In [ ]:
import requests
import pandas as pd

# Google Places API key
api_key = GOOGLE_API_KEY

# URL template for the nearby places API
url_template = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius=1500&type=restaurant&keyword=breakfast&key={}"


# Function to fetch nearby places
def fetch_nearby_places(location, api_key):
    url = url_template.format(location["lat"], location["lng"], api_key)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()["results"]
    return None


# Iterate over each row in the DataFrame and fetch nearby breakfast places
for index, row in df.iterrows():
    place_id = row["Place ID"]  # Assuming corrected column name
    response = requests.get(
        f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&fields=geometry&key={api_key}"
    )
    if response.status_code == 200:
        location = response.json()["result"]["geometry"]["location"]
        nearby_places = fetch_nearby_places(location, api_key)
        if nearby_places:
            # Print the original Place ID and the nearby breakfast places
            print(f"Original Place ID: {place_id}")
            for place in nearby_places[:2]:
                print(f"Nearby Place: {place['name']}")
                print(f"Place ID: {place['place_id']}")
                # Fetch and print details of the nearby breakfast place
                url = f'https://places.googleapis.com/v1/places/{place["place_id"]}/?fields=id,displayName,location,primaryType,editorialSummary,websiteUri,googleMapsUri,formattedAddress,businessStatus,userRatingCount,priceLevel,rating,goodForChildren&key={api_key}'
                response = requests.get(url)
                if response.status_code == 200:
                    print(response.json())
        else:
            print(f"No nearby found for Place ID: {place_id}")
    else:
        print(f"Failed to fetch geometry for Place ID: {place_id}")